In [1]:
import streamlit as st
import json
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import geopandas as gpd
import geojson
import folium
import numpy as np
import datetime
from shapely.geometry import Polygon, mapping
from calendar import month
from calendar import month_name as mn
from matplotlib import animation
import seaborn as sns



dataset = 'TILL6022_Emission_Dataset.csv'
df = pd.read_csv(dataset, delimiter=',', encoding='ISO-8859-1') #,parse_dates = ['date']
country_bb = pd.read_csv('country_bb.csv', delimiter=',', encoding='ISO-8859-1')
# country_bb.set_index('country', inplace=True)
#df.set_index('country', inplace=True)
#sf = df[df.date == '2019-01-01']
# vf = df[df.sector == 'Total']
df.date = pd.to_datetime(df.date)
df_filt = df.groupby(['country','sector']).resample('MS',on='date').sum()    #D for Day, MS for month, QS to quarter, Y for year
df_filt = df_filt.reset_index()
df_filt.date = np.datetime_as_string(df_filt.date, unit='D')
df_filt.dtypes


country     object
sector      object
date        object
co2        float64
dtype: object

Useful commands for geopandas
#ar = gdf[gdf.ADMIN == 'Aruba'] #To find the country Aruba
#print(ar.area)                 #Print area of country Aruba
# gdf.ADMIN['Aruba']
#counties['features'][3]['geometry'] access coordinates and type

In [2]:
# with open("B:\Documenten\TU_Delft\Master\Module_1\Python\Assignment\TIL6022\countries.geojson") as f:
#     counties = geojson.load(f)
# gdf = gpd.GeoDataFrame.from_features(counties["features"], crs=4326)
# mask = gdf.area >2    # Function to remove smaller countries to speed up 
# gdf_selec = gdf.loc[mask]
# gdf_selec.head()
# ar = gdf[gdf.ADMIN == 'China']['geometry'] # access geometry data of China

#counties['features'][3]['geometry'] access coordinates and type
#counties['features']['properties']
with open("B:\Documenten\TU_Delft\Master\Module_1\Python\Assignment\TIL6022\countries_simp.geojson") as f:
    counties = json.load(f)
with open("B:\Documenten\TU_Delft\Master\Module_1\Python\Assignment\TIL6022\countries_simp.geojson") as f:
    counties = json.load(f)
# gdf = gpd.GeoDataFrame.from_features(counties["features"], crs=4326)


In [3]:
# dept_geo = [0]*len(counties['features'])

# Get data from dataset (using json.loads to transform string into dict for geoshape column)
# cd_depts = gdf.ADMIN.tolist()
# dept_geo[0] = counties['features'][0]['geometry']['coordinates']
counties['features'][0]

{'type': 'Feature',
 'geometry': None,
 'properties': {'ADMIN': 'Aruba', 'ISO_A3': 'ABW'}}

In [40]:
shapes = [0]*len(counties['features'])
for feature in range(len(counties['features'])):
  shapes[feature] = counties['features'][feature]['geometry']['coordinates']
  # for cood in range(len(feature['geometry']['coordinates'])):
    # shapes[cood] = feature['geometry'][cood]['coordinates']
cd_depts = gdf.ADMIN.tolist()
for cd_dept, shape in zip(cd_depts, shapes):
    
    # Mapping information from data to GeoJSON file
    dept_geo.append({
        'type': 'Feature',
        'geometry': shape,
        'id':cd_dept
    })

# Encapsulate dept_geo in a real GeoJSON formatted file
dept_geo_ok = {'type': 'FeatureCollection', 'features': dept_geo}

In [24]:
# import pandas as pd
# import plotly
# import numpy as np


# from urllib.request import urlopen
# import json
# with urlopen('https://opendata.arcgis.com/datasets/c572ffec290b42768f80e43d31ac53a6_0.geojson') as response:
#     counties = json.load(response)
    
# df = pd.read_excel('testdata.xlsx',dtype={"cty19cd": str})
# df.columns=['CODE','NAME','VALUE']

# import plotly.express as px

# fig = px.choropleth(df, geojson=counties, locations='CODE', featureidkey="properties.cty19cd", color='VALUE',
#                            color_continuous_scale="Inferno",
#                             scope="europe")

# fig.update_geos( fitbounds="locations", visible=True)
# fig.write_html('maptest.html',auto_open=True)

In [25]:
# map = folium.Map(
#     location=[
#         gdf_selec.geometry.bounds.miny.mean(),
#         gdf_selec.geometry.bounds.minx.mean()
#     ],
#     zoom_start=1
# )

# gjson = folium.features.GeoJson(
#     gdf_selec,
# ).add_to(map)
# map

# Encapsulate dept_geo in a real GeoJSON formatted file
dept_geo = []

# Get data from dataset (using json.loads to transform string into dict for geoshape column)
cd_depts = gdf.ADMIN.tolist()
shapes = gdf.geometry
cd = mapping(shapes)
# # Iterate over lines
for cd_dept, shape in zip(cd_depts, shapes):
    
    # Mapping information from data to GeoJSON file
    dept_geo.append({
        'type': 'Feature',
        'geometry': shape,
        'id':cd_dept
    })

# Encapsulate dept_geo in a real GeoJSON formatted file
dept_geo_ok = {'type': 'FeatureCollection', 'features': dept_geo}


NameError: name 'gdf' is not defined

In [6]:
# fig = px.choropleth(vf, geojson=gdf_selec, color="co2",
#                     locations="ISO-A3", featureidkey="properties.district",
#                     projection="mercator"
#                    )
# fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()
given_time = st.selectbox('Select time period to visualise', ['Day','Month','Quartile','Year'])
if given_time == 'Day':
    date_filt = 'D'
elif given_time == 'Month':
    date_filt = 'MS'
elif given_time == 'Quartile':
    date_filt = 'QS'
elif given_time == 'Year':
    date_filt = 'Y'

given_sector = st.selectbox('Select sector to visualise', df_filt.sector.unique())

default_country = int(np.where(df['country'].unique() == 'WORLD')[0])
given_country = st.selectbox('Select country to visualise', df_filt.country.unique(), index = default_country)
default_sector = int(np.where(df['sector'].unique() == 'Total')[0])
given_sector = st.selectbox('Select sector to visualise', df_filt.sector.unique(), index = default_sector)
# given_country = "China"
# country_bb.country.index(given_country)
# country_bb.country[(country_bb.country.values==given_country).any(0)]

given_sector = 'International Shipping'
given_country = 'US'
if (given_country == 'WORLD' or given_country == 'ROW' or given_country == 'EU27 & UK'):
    latrange = [-90,90]
    lonrange = [-180,180]
elif given_country == 'US':
    given_country = 'United States'
    given_country_bb = country_bb[country_bb.country == given_country]
    latrange = [float(given_country_bb['latmin']),float(given_country_bb['latmax'])]
    lonrange = [float(given_country_bb['lonmin']),float(given_country_bb['lonmax'])]
else:
    given_country_bb = country_bb[country_bb.country == given_country]
    latrange = [float(given_country_bb['latmin']),float(given_country_bb['latmax'])]
    lonrange = [float(given_country_bb['lonmin']),float(given_country_bb['lonmax'])]


Write something about the zoom function and the implemenetation of geojson files and why they are not used.

In [8]:
df_filt[df_filt.sector == given_sector].given_country

AttributeError: 'DataFrame' object has no attribute 'given_country'

In [7]:

fig = px.choropleth(df_filt[df_filt.sector == given_sector], 
                    # geojson= counties,
                    # featureidkey='ISO_A3',
                    locations= 'country',
                    locationmode= "country names",
                    color='co2',
                    #hover_name="country",
                    animation_frame= 'date'
                    #zoom=2,
)
fig.update_geos(lataxis_range=latrange, lonaxis_range=lonrange)  
fig.update_layout(
    title_text = "CO2 emmissions transport sector",
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection_type = "equirectangular",
    ),
    
                    #color_continuous_scale="Blues_r
    height=1000,
    width  = 1080
    #margin={"r":0,"t":0,"l":0,"b":0},
)

fig.show()

In [14]:
# filter dataset on Transport sectors: 'International Shipping' or 'International Aviation' or 'Domestic Aviation' or 'Ground Transport'
sectors = ['International Shipping', 'International Aviation', 'Domestic Aviation', 'Ground Transport']
Transport_Sector =  df_filt[df_filt.sector.isin(sectors)]

Transport_not_World = Transport_Sector[Transport_Sector.country != 'WORLD'].groupby(['country', 'date']).sum()

#reset index for optimal dataset and split date into years

abc = Transport_not_World.reset_index(['country', 'date'])
abc[["year", "month", "day"]] = abc["date"].str.split("-", expand = True)
abc['Date2'] = '2019' + '-' + abc['month'].astype(str) + '-' + abc['day']

fig = go.Figure()
# add trace
Years = ['2019', '2020']
Countries = ['UK', 'US', 'China', 'EU27 & UK', 'France', 'Germany', 'India', 'Italy', 'Japan', 'Russia', 'Spain']
months = list(abc.month.unique())

for k in Countries:
    for i in Years:
        z = abc[(abc.country == k) & (abc.year == i)]
        lop = fig.add_trace(
                go.Scatter(x=list(z.Date2),
                        y=list(z.co2),
                        name= k + i,
                        visible =True))


list_updatemenus = []

for n, country in enumerate(Countries):
    visible = [False] * 2 * len(Countries)
    visible[2*n] = True
    visible[2*n + 1] = True
    temp_dict = dict(label = str(country),
                 method = 'update',
                 args = [{'visible': visible},
                         {'title': ('CO2 emissions comparison 2019 versus 2020 in %s'% country)}])
    list_updatemenus.append(temp_dict)
    
Transport_not_World = Transport_Sector[Transport_Sector.country != 'WORLD'].groupby(['country', 'date']).sum()
#reset index for optimal dataset and split date into years

abc = Transport_not_World.reset_index(['country', 'date'])
abc[["year", "month", "day"]] = abc["date"].str.split("-", expand = True)

# create dataframe for sector

Transport_Sector[["year", "month", "day"]] = Transport_Sector["date"].str.split("-", expand = True)
Transport_Sector['Date2'] = '2019' + '-' + Transport_Sector['month'].astype(str) + '-' + Transport_Sector['day']
Transport_Sector2 = Transport_Sector[(Transport_Sector.year == '2019') | (Transport_Sector.year == '2020')]

Transport_not_World = Transport_Sector[Transport_Sector.country != 'WORLD'].groupby(['country', 'date']).sum()

#reset index for optimal dataset and split date into years

abc = Transport_not_World.reset_index(['country', 'date'])
abc[["year", "month", "day"]] = abc["date"].str.split("-", expand = True)
abc['Date2'] = '2019' + '-' + abc['month'].astype(str) + '-' + abc['day']

# add trace
Years = ['2019', '2020']
Countries = ['UK', 'US', 'China', 'EU27 & UK', 'France', 'Germany', 'India', 'Italy', 'Japan', 'Russia', 'Spain']
sectors = ['International Aviation', 'Domestic Aviation', 'Ground Transport']

fig = make_subplots(rows = 4, 
                    cols = 1,
                    subplot_titles = ('Total', 'International Aviation', 'Domestic Aviation', 'Ground Transport'))

for k in Countries:
    for i in Years:
        z = abc[(abc.country == k) & (abc.year == i)]
        Int_Av = Transport_Sector2[(Transport_Sector2.country == k) & (Transport_Sector2.year == i) & (Transport_Sector2.sector == 'International Aviation')]
        Do_Av = Transport_Sector2[(Transport_Sector2.country == k) & (Transport_Sector2.year == i) & (Transport_Sector2.sector == 'Domestic Aviation')]
        Gr_Trans = Transport_Sector2[(Transport_Sector2.country == k) & (Transport_Sector2.year == i) & (Transport_Sector2.sector == 'Ground Transport')]
        lop = fig.add_traces(
                [
                    go.Scatter(x=list(z.Date2),
                        y=list(z.co2),
                        name= k + i,
                        visible = False),        
                    go.Scatter(x=list(Int_Av.Date2),
                            y=list(Int_Av.co2),
                            name= 'International Aviation' + k + i,
                            visible = False),
                    go.Scatter(x=list(Do_Av.Date2),
                            y=list(Do_Av.co2),
                            name= 'Domestic Aviation' + k + i,
                            visible = False),
                    go.Scatter(x=list(Gr_Trans.Date2),
                            y=list(Gr_Trans.co2),
                            name= 'Ground Transport' + k + i,
                            visible = False)
                ], rows=list(range(1,5)), cols=[1, 1,1,1])

fig.update_layout(
    height = 1500
)

list_updatemenus = []


for n, country in enumerate(Countries):
    visible = [False] * 2 * len(Countries)
    visible[2*n] = True
    visible[2*n + 1] = True
    temp_dict = dict(label = str(country),
                 method = 'update',
                 args = [{'visible': visible},
                         {'title': ('CO2 emissions comparison 2019 versus 2020 by sector in %s'% country)}])
    list_updatemenus.append(temp_dict)

fig.update_layout(
    updatemenus=[
        dict(
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0,
            xanchor="left",
            y=1.04,
            yanchor="top"
        ),
    ]
)
fig.update_layout(updatemenus=list([dict(buttons= list_updatemenus)]), xaxis_title = 'Date', yaxis_title = 'CO2 emissions x 1000 (ppm)')
fig.update_layout(title_text = 'CO2 emissions comparison 2019 versus 2020')
fig.update_layout(xaxis=dict(tickformat="%d-%B"),
                 xaxis2=dict(tickformat="%d-%B"),
                 xaxis3=dict(tickformat="%d-%B"),
                 xaxis4=dict(tickformat="%d-%B")
                 )
fig['layout']['yaxis2']['title']='CO2 emissions x 1000 (ppm)'
fig['layout']['yaxis3']['title']='CO2 emissions x 1000 (ppm)'
fig['layout']['yaxis4']['title']='CO2 emissions x 1000 (ppm)'
fig['layout']['xaxis2']['title']='Date'
fig['layout']['xaxis3']['title']='Date'
fig['layout']['xaxis4']['title']='Date'


fig.show()


C:\Users\timtr\AppData\Local\Temp\ipykernel_28824\1579841447.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\timtr\AppData\Local\Temp\ipykernel_28824\1579841447.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\timtr\AppData\Local\Temp\ipykernel_28824\1579841447.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa